In [1]:
from google.colab import drive
import os
import sys
drive.mount('/content/gdrive/', force_remount=True)
outdir = "/content/gdrive/MyDrive/project"
assert not os.system("mkdir -p "+outdir)
sys.path.insert(0, outdir)

Mounted at /content/gdrive/


In [2]:
result = "/content/gdrive/MyDrive/result"
#import transformers
#from datasets import load_dataset, load_metric, load_from_disk
from datasets import load_metric, load_from_disk
#import datasets
import random
import pandas as pd
from IPython.display import display, HTML
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer,AutoTokenizer
import nltk
import numpy as np
nltk.download('punkt', download_dir=result)
nltk.data.path.append(result)
raw_datasets = load_dataset("ccdv/arxiv-summarization", cache_dir=result)
metric = load_metric("rouge")

[nltk_data] Downloading package punkt to
[nltk_data]     /content/gdrive/MyDrive/result...
[nltk_data]   Package punkt is already up-to-date!


Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

In [3]:
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

def preprocess_function(examples, max_input_length=4096, max_target_length = 256):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["abstract"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

# raw_datasets["train"][0]
# show_random_elements(raw_datasets["train"])
# fake_preds = ["hello there", "general kenobi"]
# fake_labels = ["hello there", "general kenobi"]
# metric.compute(predictions=fake_preds, references=fake_labels)
# tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
# tokenized_datasets.save_to_disk(result+"/tokenized_datasets")
#["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:

In [ ]:
model_checkpoint = "t5-small"
model_checkpoint = result+"/t5-small-finetuned-arxiv/checkpoint-8000"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, cache_dir=result)
#tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
#tokenized_datasets.save_to_disk(result+"/arxiv_tokenized_datasets")
tokenized_datasets = load_from_disk(result+"/arxiv_tokenized_datasets")
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [10]:
args = Seq2SeqTrainingArguments(
    f"{result}/t5-small-finetuned-arxiv",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=1,
    eval_steps = 4000,
    predict_with_generate=True,
    save_steps = 2000,
    fp16=True)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp fp16 backend


In [12]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: abstract, article.
***** Running Evaluation *****
  Num examples = 6436
  Batch size = 4


{'eval_gen_len': 19.0,
 'eval_loss': 2.042651891708374,
 'eval_rouge1': 12.6477,
 'eval_rouge2': 5.4532,
 'eval_rougeL': 10.8277,
 'eval_rougeLsum': 11.9012,
 'eval_runtime': 2591.69,
 'eval_samples_per_second': 2.483,
 'eval_steps_per_second': 0.621}

In [17]:
#trainer.evaluate()
outputs = trainer.predict(tokenized_datasets["test"])
output_str = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)
raw_datasets["test"].to_csv(result+"/arxiv_test.csv")
pd.DataFrame(output_str).to_csv(result+"/arxiv_pred.csv")

The following columns in the test set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: abstract, article.
***** Running Prediction *****
  Num examples = 6440
  Batch size = 4


Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]